In [264]:
import pandas as pd
import numpy as np
import json
import ast
import importlib
import validation

# Reload the functions module
importlib.reload(validation)

<module 'validation' from 's:\\Courses\\Data Science\\0. Henry\\M7 - Labs - Project MLOps _ steam\\game-recommendation-system\\data_transformation\\validation.py'>

## User reviews

### Creating a DataFrame

Creating a DataFrame from the australian_user_reviews JSON file

In [2]:
# Open JSON file and load data in a variable
review_data = []
with open(r"./raw_data/australian_user_reviews.json", encoding="MacRoman") as file:
    for line in file.readlines():
        review_data.append(ast.literal_eval(line))

In [3]:
# Create a DataFrame from variable
df_user_reviews = pd.DataFrame(review_data)
df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


### Preparing the user reviews data

#### Checking Null values

In [4]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


No null values in the dataset

#### Checking Duplicate values

In [5]:
# Call the function to check for duplicates in the "user_url" column
duplicates_user_url = validation.check_duplicates_summary(df_user_reviews, "user_url")

# Print the summary
print(duplicates_user_url)

                                      Duplicated Value  Occurrences
0    http://steamcommunity.com/profiles/76561198051...            3
2    http://steamcommunity.com/profiles/76561198045...            3
3    http://steamcommunity.com/profiles/76561198100...            3
4            http://steamcommunity.com/id/blablabla174            3
1    http://steamcommunity.com/profiles/76561198027...            3
..                                                 ...          ...
104    http://steamcommunity.com/id/syntheticflyingfox            2
103           http://steamcommunity.com/id/TandyTandon            2
102  http://steamcommunity.com/id/wasntthereanumber...            2
101               http://steamcommunity.com/id/norsktf            2
308         http://steamcommunity.com/id/OfficialvnmZz            2

[309 rows x 2 columns]


Duplicate user_url values are removed, as they will affect future measurements

#### Removing Duplicate values

In [6]:
df_user_reviews_1 = validation.remove_duplicates(df_user_reviews, "user_url")

Removed 314 duplicate rows based on 'user_url' column.


In [7]:
# Verifiying no duplicate values left

# Call the function to check for duplicates in the "user_id" column
duplicates_user_id = validation.check_duplicates_summary(df_user_reviews_1, "user_id")

# Print the summary
print(duplicates_user_id)


Empty DataFrame
Columns: [Duplicated Value, Occurrences]
Index: []


#### Checking None values

In [8]:
# Call the function to check for none values in the "user_id" column
none_user_id = validation.check_none_values(df_user_reviews_1, "user_id")

# Print the summary
print(none_user_id)

{'column_name': 'user_id', 'total_rows': 25485, 'num_none_values': 0, 'rows_below_threshold': None}


#### Unnesting "reviews" data

In [9]:
df_user_reviews_1["reviews"][4]

[{'funny': '3 people found this review funny',
  'posted': 'Posted April 15, 2014.',
  'last_edited': '',
  'item_id': '211420',
  'helpful': '35 of 43 people (81%) found this review helpful',
  'recommend': True,
  'review': 'Git gud'},
 {'funny': '1 person found this review funny',
  'posted': 'Posted December 23, 2013.',
  'last_edited': '',
  'item_id': '211820',
  'helpful': '12 of 16 people (75%) found this review helpful',
  'recommend': True,
  'review': "It's like Terraria, you play for 9 hours straight, get endgame armour then stop playing until the next update."},
 {'funny': '2 people found this review funny',
  'posted': 'Posted March 14, 2014.',
  'last_edited': '',
  'item_id': '730',
  'helpful': '5 of 5 people (100%) found this review helpful',
  'recommend': True,
  'review': 'Hold shift to win, Hold CTRL to lose.'},
 {'funny': '',
  'posted': 'Posted July 11, 2013.',
  'last_edited': '',
  'item_id': '204300',
  'helpful': 'No ratings yet',
  'recommend': True,
  'rev

In [10]:
# Verifying the data types of "reviews" data
reviews_data_type = validation.column_data_types_summary(df_user_reviews_1, "reviews")
reviews_data_type

{'column_name': 'reviews', 'data_type_counts': {"<class 'list'>": 25485}}

Knowing that all values of the columns are list, the function "extract_values" will extract each value, based on a key parameter, and store it on a new list in different columns

In [11]:
# Asigning values to variables
column_name = "reviews"
keys = ["funny", "posted", "last_edited", "item_id", "helpful", "recommend", "review"]
new_columns = ["funny", "posted", "last_edited", "item_id", "helpful", "recommend", "review"]

In [12]:
# Creating a copy of the dataframe
df_user_reviews_2 = df_user_reviews_1.copy()

# Ectracting nested values from reviews
validation.extract_values(df_user_reviews_2, column_name, keys, new_columns)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","[, , ]","[Posted November 5, 2011., Posted July 15, 201...","[, , ]","[1250, 22200, 43110]","[No ratings yet, No ratings yet, No ratings yet]","[True, True, True]",[Simple yet with great replayability. In my op...
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...","[, , ]","[Posted June 24, 2014., Posted September 8, 20...","[, , ]","[251610, 227300, 239030]",[15 of 20 people (75%) found this review helpf...,"[True, True, True]",[I know what you think when you see this title...
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...","[, , , , , ]","[Posted February 3., Posted December 4, 2015.,...","[, Last edited December 5, 2015., , , , ]","[248820, 370360, 237930, 263360, 107200, 224500]","[No ratings yet, No ratings yet, No ratings ye...","[True, True, True, True, True, True]",[A suitably punishing roguelike platformer. W...
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...","[, , , , , ]","[Posted October 14, 2013., Posted July 28, 201...","[, , , , , ]","[250320, 20920, 204100, 224600, 207610, 108710]",[2 of 2 people (100%) found this review helpfu...,"[True, True, True, True, True, True]",[This game... is so fun. The fight sequences h...
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...","[3 people found this review funny, 1 person fo...","[Posted April 15, 2014., Posted December 23, 2...","[, , , ]","[211420, 211820, 730, 204300]",[35 of 43 people (81%) found this review helpf...,"[True, True, True, True]","[Git gud, It's like Terraria, you play for 9 h..."
...,...,...,...,...,...,...,...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la...",[],[Posted May 31.],[],[261030],[0 of 1 people (0%) found this review helpful],[True],[I cried in the end its so sadding ]'; I wish ...
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l...",[],[Posted June 17.],[],[730],[0 of 1 people (0%) found this review helpful],[True],[Gra naprawdƒô fajna.Ale jest kilka rzeczy do ...
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',...",[1 person found this review funny],[Posted June 23.],[],[570],[1 of 1 people (100%) found this review helpful],[True],[Well Done]
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l...","[, , , ]","[Posted July 21., Posted July 10., Posted July...","[, , , ]","[233270, 130, 70, 362890]","[No ratings yet, No ratings yet, No ratings ye...","[True, True, True, True]",[this is a very fun and nice 80s themed shoote...


#### Transforming to numeric type

This will facilitate the analysis in EDA and the creation of the API functions

##### Column "item_id"

In [259]:
# Verifying the type of each value of the ""item_id column
item_id_type_summary = validation.column_data_types_summary(df_user_reviews_2, "item_id")
item_id_type_summary

{'column_name': 'item_id', 'data_type_counts': {"<class 'list'>": 25485}}

In [255]:
# Verifying data type of values inside "item_id"
print(df_user_reviews_2["item_id"][0])
print(type(df_user_reviews_2["item_id"][0]))

print(df_user_reviews_2["item_id"][0][0])
print(type(df_user_reviews_2["item_id"][0][0]))

['1250', '22200', '43110']
<class 'list'>
1250
<class 'str'>


In [248]:
# Creating a copy of the dataframe
df_user_reviews_3 = df_user_reviews_2.copy()

# Transforming to numeric type using "convert_column_to_numeric" function
validation.convert_column_to_numeric(df_user_reviews_3, "item_id")

{'total_rows': 25485, 'column_name': 'item_id', 'report': {}}

In [256]:
# Verifying data type of values inside "item_id" lists
print(df_user_reviews_3["item_id"][0][0])
print(type(df_user_reviews_3["item_id"][0][0]))

1250
<class 'numpy.int64'>


#### Transforming to date type

This will facilitate the analysis in EDA and the creation of the API functions

##### Column "posted"

In [260]:
# Verifying the type of each value of the ""item_id column
posted_type_summary = validation.column_data_types_summary(df_user_reviews_3, "posted")
posted_type_summary

{'column_name': 'posted', 'data_type_counts': {"<class 'list'>": 25485}}

In [265]:
# Verifying data type of values inside "posted"
print(df_user_reviews_3["posted"][0])
print(type(df_user_reviews_3["posted"][0]))

print(df_user_reviews_3["posted"][0][0])
print(type(df_user_reviews_3["posted"][0][0]))

['Posted November 5, 2011.', 'Posted July 15, 2011.', 'Posted April 21, 2011.']
<class 'list'>
Posted November 5, 2011.
<class 'str'>


In [268]:
# Creating a copy of the dataframe
df_user_reviews_4 = df_user_reviews_3.copy()

# Setting the original datetime patterns
# The "Posted %B %d." patter of the datetime values of the datframe will automatically take the year 1900,
# this will help to avoid removing date from the dataframe and for further analysis.
original_date_list = ["Posted %B %d, %Y.", "Posted %B %d."]

# Transforming to datetime type using "convert_column_to_dates" function
validation.convert_column_to_dates(df_user_reviews_4, "posted", original_date_list)

{'total_rows': 25485,
 'column_name': 'posted',
 'num_failed_conversions': 24,
 'report': {348: {6: 'Posted February 29.'},
  1548: {1: 'Posted February 29.'},
  2715: {3: 'Posted February 29.'},
  2928: {2: 'Posted February 29.'},
  3891: {3: 'Posted February 29.'},
  5129: {0: 'Posted February 29.'},
  6132: {0: 'Posted February 29.'},
  6200: {0: 'Posted February 29.'},
  9280: {0: 'Posted February 29.'},
  10652: {1: 'Posted February 29.'},
  10692: {0: 'Posted February 29.'},
  10833: {0: 'Posted February 29.'},
  10851: {0: 'Posted February 29.'},
  11617: {1: 'Posted February 29.'},
  12169: {0: 'Posted February 29.'},
  15186: {1: 'Posted February 29.'},
  15835: {2: 'Posted February 29.'},
  16397: {0: 'Posted February 29.'},
  16429: {0: 'Posted February 29.'},
  16730: {0: 'Posted February 29.'},
  20752: {0: 'Posted February 29.', 1: 'Posted February 29.'},
  22212: {0: 'Posted February 29.'},
  24464: {4: 'Posted February 29.'},
  24771: {0: 'Posted February 29.'}}}

In [270]:
# Verifying data type of values inside "item_id" lists
print(df_user_reviews_4["posted"][1600][0])
print(type(df_user_reviews_4["posted"][1600][0]))

2014-02-12
<class 'str'>


### Creating file

A csv file is created to be used in EDA

In [13]:
df_user_reviews_2.columns

Index(['user_id', 'user_url', 'reviews', 'funny', 'posted', 'last_edited',
       'item_id', 'helpful', 'recommend', 'review'],
      dtype='object')

In [14]:
# Necessary data
# all columns except "reviews" for the nested data

necessary_columns = ["user_id", "user_url", "item_id", "posted", "last_edited", "helpful", "recommend", "funny", "review"]

# Create a new DataFrame with only the selected columns
user_reviews_dataframe = df_user_reviews_2[necessary_columns]

# Save the selected DataFrame to a CSV file
user_reviews_dataframe.to_csv('../eda/datasets/user_reviews.csv', index=False)

## User Items

### Creating a DataFrame

Creating a DataFrame from the australian_users_items JSON file

In [15]:
# Open JSON file and load data in variable
items_data = []
with open(r"./raw_data/australian_users_items.json", encoding="MacRoman") as file:
    for line in file.readlines():
        items_data.append(ast.literal_eval(line))

In [16]:
# Create a DataFrame from variable
df_user_items = pd.DataFrame(items_data)
df_user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


### Preparing the users items data

#### Checking Null values

In [17]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


No Null values found

#### Checking Duplicate values

In [18]:
# Call the function to check for duplicates in the "user_url" column
duplicates_steam_id = validation.check_duplicates_summary(df_user_items, "steam_id")

# Print the summary
print(duplicates_steam_id)

      Duplicated Value  Occurrences
0    76561198094973305            3
6    76561198085989695            3
1    76561198072861800            3
9    76561198051777058            3
8    76561198081666970            3
..                 ...          ...
231  76561198026742195            2
232  76561198035336388            2
233  76561198071790027            2
234  76561198073621277            2
673  76561198086134170            2

[674 rows x 2 columns]


Duplicate steam_id values are removed, as they will affect future measurements

#### Removing Duplicate values

In [19]:
df_user_items_1 = validation.remove_duplicates(df_user_items, "steam_id")

Removed 685 duplicate rows based on 'steam_id' column.


In [20]:
# Verifiying no duplicate values left

# Call the function to check for duplicates in the "steam_id" column
duplicates_steam_id = validation.check_duplicates_summary(df_user_items_1, "steam_id")

# Print the summary
print(duplicates_steam_id)

Empty DataFrame
Columns: [Duplicated Value, Occurrences]
Index: []


#### Checking None values

In [21]:
# Call the function to check for none values in the "steam_id" column
none_steam_id = validation.check_none_values(df_user_items_1, "steam_id")

# Print the summary
print(none_steam_id)

{'column_name': 'steam_id', 'total_rows': 87625, 'num_none_values': 0, 'rows_below_threshold': None}


No None values found

#### Checking data type of "items count"

In [45]:
# Call the function to check data type of values in the "items_count" column
user_items_count_summary = validation.column_data_types_summary(df_user_items_1, "items_count")
user_items_count_summary

{'column_name': 'items_count', 'data_type_counts': {"<class 'int'>": 87625}}

#### Unesting "items" data

In [41]:
df_user_items_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87625 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      87625 non-null  object
 1   items_count  87625 non-null  int64 
 2   steam_id     87625 non-null  object
 3   user_url     87625 non-null  object
 4   items        87625 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.0+ MB


In [46]:
# Verifying the data types of "items" column
items_data_type = validation.column_data_types_summary(df_user_items_1, "items")
items_data_type

{'column_name': 'items', 'data_type_counts': {"<class 'list'>": 87625}}

In [60]:
# Asigning values to variables
items_column_name = "items"
items_keys = ["item_id", "item_name", "playtime_forever", "playtime_2weeks"]
items_new_columns = ["item_id", "item_name", "playtime_forever", "playtime_2weeks"]

In [61]:
# Creating a copy of the dataframe
df_user_items_2 = df_user_items_1.copy()

# Ectracting nested values from items
validation.extract_values(df_user_items_2, items_column_name, items_keys, items_new_columns)

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[Counter-Strike, Team Fortress Classic, Day of...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[Counter-Strike, Counter-Strike: Condition Zer...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[Red Orchestra: Ostfront 41-45, Mare Nostrum, ...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[Counter-Strike, Team Fortress Classic, Day of...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[Day of Defeat: Source, Team Fortress Classic,...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla...","[413850, 413851, 413852, 413853, 413854, 41385...","[CS:GO Player Profiles, CS:GO Player Profiles:...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania...","[11020, 6370, 13140, 51100, 107900, 113400, 20...","[TrackMania Nations Forever, Bloodline Champio...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[],[],[],[],[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'...","[304930, 227940, 346330, 373330, 388490, 52157...","[Unturned, Heroes & Generals, BrainBread 2, Al...","[677, 43, 0, 0, 3, 4, 3]","[677, 43, 0, 0, 3, 4, 3]"


In [ ]:
df_user_items_2["item_id"][0]

### Creating file

Creating a csv file to be used in EDA

In [75]:
df_user_items_2.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items', 'item_id',
       'item_name', 'playtime_forever', 'playtime_2weeks'],
      dtype='object')

In [76]:
# Necessary data
# all columns except "items" for the nested data

items_necessary_columns = ["user_id", "steam_id", "user_url", "items_count", "item_id", "item_name", "playtime_forever", "playtime_2weeks"]

# Create a new DataFrame with only the selected columns
user_items_dataframe = df_user_items_2[items_necessary_columns]

# Save the selected DataFrame to a CSV file
user_items_dataframe.to_csv('../eda/datasets/user_items.csv', index=False)

## Steam Games

### Creating a DataFrame

Creating a DataFrame from the output_steam_games JSON file

In [22]:
# Open JSON file and load data in variable
games_data = []
with open("./raw_data/output_steam_games.json", encoding="MacRoman") as file:
    for line in file:
        games_data.append(json.loads(line))

In [23]:
# Create a DataFrame from variable
df_games_data = pd.DataFrame(games_data)
df_games_data

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


### Preparing the steam games data

#### Checking Duplicate values

In [24]:
# Call the function to check for duplicates in the "id" column
duplicates_games_id = validation.check_duplicates_summary(df_games_data, "id")

# Print the summary
print(duplicates_games_id)

  Duplicated Value  Occurrences
0           612880            2


Duplicate id values are removed, as they will affect future measurements

#### Removing Duplicate values

In [25]:
df_games_data_1 = validation.remove_duplicates(df_games_data, "id")

Removed 88312 duplicate rows based on 'id' column.


In [26]:
# Verifiying no duplicate values left

# Call the function to check for duplicates in the "id" column
duplicates_games_id = validation.check_duplicates_summary(df_games_data_1, "id")

# Print the summary
print(duplicates_games_id)

Empty DataFrame
Columns: [Duplicated Value, Occurrences]
Index: []


In [27]:
df_games_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24081 non-null  object
 1   genres        28850 non-null  object
 2   app_name      32131 non-null  object
 3   title         30083 non-null  object
 4   url           32132 non-null  object
 5   release_date  30066 non-null  object
 6   tags          31970 non-null  object
 7   reviews_url   32132 non-null  object
 8   specs         31463 non-null  object
 9   price         30755 non-null  object
 10  early_access  32132 non-null  object
 11  id            32132 non-null  object
 12  developer     28834 non-null  object
dtypes: object(13)
memory usage: 3.4+ MB


#### Checking None values

In [28]:
# Call the function to check for none values in the "id" column
# the function receives additional parameter to check if the dataframe has mutiple 
# columns with None values
none_games_id = validation.check_none_values(df_games_data_1, "id", True, 4)

# Print the summary
print(none_games_id)

{'column_name': 'id', 'total_rows': 32133, 'num_none_values': 1, 'rows_below_threshold': 2062}


#### Removing None values

Rows that have None value in the "price" and "id" column are removed, because these values are important when you want to present the price in the API

In [29]:
# Call the function to remove none values in the "id" column
df_games_data_2 = validation.remove_none_values(df_games_data_1, "id")

# Call the function to remove none values in the "price" column
df_games_data_2 = validation.remove_none_values(df_games_data_1, "price")

In [30]:
df_games_data_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30755 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     22982 non-null  object
 1   genres        27620 non-null  object
 2   app_name      30755 non-null  object
 3   title         28823 non-null  object
 4   url           30755 non-null  object
 5   release_date  28819 non-null  object
 6   tags          30625 non-null  object
 7   reviews_url   30755 non-null  object
 8   specs         30100 non-null  object
 9   price         30755 non-null  object
 10  early_access  30755 non-null  object
 11  id            30755 non-null  object
 12  developer     27599 non-null  object
dtypes: object(13)
memory usage: 3.3+ MB


#### Transforming to numeric type

To have better performance in the API

##### "id" column

In [31]:
# Call the function to summarize the data types of the "id" values
id_summary = validation.column_data_types_summary(df_games_data_2, "id")
id_summary

{'column_name': 'id', 'data_type_counts': {"<class 'str'>": 30755}}

In [32]:
# Making a copy of the dataframe
df_games_data_3 = df_games_data_2.copy()

In [33]:
# Calling the function to transform "id" values into numeric type
id_transformation = validation.transform_to_numeric(df_games_data_3, "id")
id_transformation

{'total_values': 30755,
 'numeric_count': 30755,
 'non_numeric_count': 0,
 'stored_info': []}

##### "price" column

In [34]:
# Call the function to summarize the data types of the "price" values
price_summary = validation.column_data_types_summary(df_games_data_3, "price")
price_summary

{'column_name': 'price',
 'data_type_counts': {"<class 'float'>": 28845, "<class 'str'>": 1910}}

In [35]:
# Calling the function to transform "price" values into numeric type
price_transformation = validation.transform_to_numeric(df_games_data_3, "price")
price_transformation

{'total_values': 30755,
 'numeric_count': 30734,
 'non_numeric_count': 21,
 'stored_info': [{'index': 89230, 'value': 'Free Demo'},
  {'index': 89231, 'value': 'Free Demo'},
  {'index': 89279, 'value': 'Play for Free!'},
  {'index': 89291, 'value': 'Play for Free!'},
  {'index': 90715, 'value': 'Install Now'},
  {'index': 91181, 'value': 'Play WARMACHINE: Tactics Demo'},
  {'index': 91624, 'value': 'Free Mod'},
  {'index': 92142, 'value': 'Install Theme'},
  {'index': 92228, 'value': 'Third-party'},
  {'index': 93899, 'value': 'Free HITMAN™ Holiday Pack'},
  {'index': 102257, 'value': 'Free Mod'},
  {'index': 111044, 'value': 'Play the Demo'},
  {'index': 113310, 'value': 'Starting at $499.00'},
  {'index': 113311, 'value': 'Starting at $449.00'},
  {'index': 115214, 'value': 'Free to Try'},
  {'index': 116624, 'value': 'Free Mod'},
  {'index': 116625, 'value': 'Free Mod'},
  {'index': 117676, 'value': 'Free Movie'},
  {'index': 118690, 'value': 'Free to Use'},
  {'index': 119220, 'val

Manually converting the rest of the free game values

In [36]:
idx_free_games = [89230, 89231, 89279, 89291, 90715, 91181, 91624, 92142, 92228, 93899, 102257, 111044, 115214, 116624, 116625, 117676, 118690, 119220, 120148]

for idx in idx_free_games:
    df_games_data_3["price"][idx] = 0

Manually converting non free game values

In [37]:
# Calling the function to see summary of data values
price_transformation = validation.transform_to_numeric(df_games_data_3, "price")
price_transformation

{'total_values': 30755,
 'numeric_count': 30753,
 'non_numeric_count': 2,
 'stored_info': [{'index': 113310, 'value': 'Starting at $499.00'},
  {'index': 113311, 'value': 'Starting at $449.00'}]}

In [38]:
idx_games = [113310, 113311]
for idx in idx_games:
    df_games_data_3["price"][idx] = 499

In [39]:
df_games_data_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30755 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     22982 non-null  object
 1   genres        27620 non-null  object
 2   app_name      30755 non-null  object
 3   title         28823 non-null  object
 4   url           30755 non-null  object
 5   release_date  28819 non-null  object
 6   tags          30625 non-null  object
 7   reviews_url   30755 non-null  object
 8   specs         30100 non-null  object
 9   price         30755 non-null  object
 10  early_access  30755 non-null  object
 11  id            30755 non-null  object
 12  developer     27599 non-null  object
dtypes: object(13)
memory usage: 4.3+ MB
